In [11]:
import pandas as pd
import hvplot.pandas
import numpy as np

In [12]:
df = pd.read_csv('callsforservice.csv')

In [13]:
with open('apikey.txt', 'r') as src:
    key = src.read()

In [14]:
df.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,CALL_RECEIVED,HUNDREDBLOCKADDR,GRID
0,201502108490,N,NO ACTION REQUIRED,647V,SUSPICIOUS PERSON IN VEHICLE,11/2/2015 9:16:27AM,154XX N 29TH ST,DF34
1,201502108749,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 9:58:57AM,96XX N 10TH AVE,CH26
2,201502108648,O,OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY),459A,BURGLARY ALARM,11/2/2015 9:42:42AM,38XX E WATKINS ST,AH36B
3,201502108757,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 10:02:11AM,36XX W CAMELBACK RD,BI20
4,201502108592,N,NO ACTION REQUIRED,900,CHECK WELFARE,11/2/2015 9:30:53AM,40XX N 44TH AVE,BG18


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2171125 entries, 0 to 2171124
Data columns (total 8 columns):
INCIDENT_NUM        int64
DISP_CODE           object
DISPOSITION         object
FINAL_RADIO_CODE    object
FINAL_CALL_TYPE     object
CALL_RECEIVED       object
HUNDREDBLOCKADDR    object
GRID                object
dtypes: int64(1), object(7)
memory usage: 132.5+ MB


In [16]:
tst = df.iloc[:500, :]
tst.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,CALL_RECEIVED,HUNDREDBLOCKADDR,GRID
0,201502108490,N,NO ACTION REQUIRED,647V,SUSPICIOUS PERSON IN VEHICLE,11/2/2015 9:16:27AM,154XX N 29TH ST,DF34
1,201502108749,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 9:58:57AM,96XX N 10TH AVE,CH26
2,201502108648,O,OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY),459A,BURGLARY ALARM,11/2/2015 9:42:42AM,38XX E WATKINS ST,AH36B
3,201502108757,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 10:02:11AM,36XX W CAMELBACK RD,BI20
4,201502108592,N,NO ACTION REQUIRED,900,CHECK WELFARE,11/2/2015 9:30:53AM,40XX N 44TH AVE,BG18


In [17]:
df['HUNDREDBLOCKADDR'] = df.HUNDREDBLOCKADDR.str.replace('XX', '00')
df.dropna(inplace=True)
df = df[df.FINAL_RADIO_CODE.isin(['961', '962', '963', '961H', '962H', '963H'])]
df.loc[:, 'CALL_RECEIVED'] = pd.to_datetime(df.CALL_RECEIVED)
df.set_index('CALL_RECEIVED', inplace=True)

In [18]:
from geopandas.tools import geocode
df.loc[:, 'address'] = df.HUNDREDBLOCKADDR.apply(lambda x: x + '., PHOENIX, AZ')

In [ ]:
geo = geocode(df['address'], api_key=key, timeout=15)

In [ ]:
# lacs = Legal accidents
#lacs = df[df.FINAL_RADIO_CODE.isin(['961', '962', '963'])]
#hitrun = df[df.FINAL_RADIO_CODE.isin(['961H', '962H', '963H'])]

In [23]:
lacs.loc[:, 'CALL_RECEIVED'] = pd.to_datetime(lacs.CALL_RECEIVED)
lacs.set_index('CALL_RECEIVED', inplace=True)

In [24]:
hitrun.loc[:, 'CALL_RECEIVED'] = pd.to_datetime(hitrun.CALL_RECEIVED)
hitrun.set_index('CALL_RECEIVED', inplace=True)

In [29]:
lacs.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,HUNDREDBLOCKADDR,GRID
CALL_RECEIVED,,,,,,,
2015-11-02 08:04:44,201502108062,A,ACCIDENT REPORT,962,ACCIDENT WITH INJURIES,N 49TH DR & W ESCUDA DR,EA17
2015-11-01 10:09:50,201502102088,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,E PARADISE LN & N 32ND ST,DF35
2015-11-01 10:35:06,201502102198,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,4000 W CAMELBACK RD,BI19A
2015-11-01 18:03:38,201502104894,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,3700 W SIERRA ST,DA20
2015-11-01 18:22:35,201502105017,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,1900 E THOMAS RD,BE31


In [46]:
from geopandas.tools import geocode
lacs.loc[:, 'address'] = lacs.HUNDREDBLOCKADDR.apply(lambda x: x + '., PHOENIX, AZ')

In [45]:
lacsgeo = geocode(lacs['address'], api_key=key, timeout=10)

GeocoderTimedOut: Service timed out

In [ ]:
hitrungeo = geocode(hitrun['HUNDREDBLOCKADDR'], api_key=key)

In [15]:
lacs.DISPOSITION.value_counts() / len(lacs) * 100

ACCIDENT REPORT                           61.465645
NO ACTION REQUIRED                        31.046012
DEPARTMENTAL REPORT                        7.037190
OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY)     0.230612
TRAFFIC CITATION                           0.220542
Name: DISPOSITION, dtype: float64

In [16]:
hitrun.DISPOSITION.value_counts() / len(hitrun) * 100

NO ACTION REQUIRED                        51.448047
DEPARTMENTAL REPORT                       24.762867
ACCIDENT REPORT                           23.230065
OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY)     0.483283
TRAFFIC CITATION                           0.075738
Name: DISPOSITION, dtype: float64

In [17]:
lacs.FINAL_CALL_TYPE.value_counts() / len(lacs) * 100

ACCIDENT NO INJURIES      81.196564
ACCIDENT WITH INJURIES    18.588937
ACCIDENT FATALITY          0.214499
Name: FINAL_CALL_TYPE, dtype: float64

In [18]:
hitrun.FINAL_CALL_TYPE.value_counts() / len(hitrun) * 100

HIT & RUN ACCIDENT NO INJURY        93.573052
HIT & RUN ACCIDENT WITH INJURIES     6.336784
HIT & RUN  FATALITY                  0.090165
Name: FINAL_CALL_TYPE, dtype: float64

In [19]:
len(hitrun) / (len(hitrun) + len(lacs))

0.21827471108731933

In [20]:
lacs.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,HUNDREDBLOCKADDR,GRID
CALL_RECEIVED,,,,,,,
2015-11-02 08:04:44,201502108062,A,ACCIDENT REPORT,962,ACCIDENT WITH INJURIES,N 49TH DR & W ESCUDA DR,EA17
2015-11-01 10:09:50,201502102088,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,E PARADISE LN & N 32ND ST,DF35
2015-11-01 10:35:06,201502102198,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,4000 W CAMELBACK RD,BI19A
2015-11-01 18:03:38,201502104894,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,3700 W SIERRA ST,DA20
2015-11-01 18:22:35,201502105017,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,1900 E THOMAS RD,BE31
